# Download Airnow Hourly AQI

Downloads Airnow's Hourly AQI files, saving them in `airnow-data/hourly-aqi/dat`.

Reports to stat.createlab.org as `Airnow Hourly AQI - Download`.

Docs for the hourly data files are here: https://docs.airnowapi.org/docs/HourlyAQObsFactSheet.pdf

In [ ]:
import json, os, dateutil, re, requests, subprocess, datetime, glob, stat

from dateutil import rrule, tz, parser

In [ ]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb


def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())


exec_ipynb('./python-utils/utils.ipynb')
exec_ipynb('./airnow-common.ipynb')

In [ ]:
# 2019-05-28 15:00:00 seems to be the earliest hourly data file available as of 2020-05-11 (data file https://files.airnowtech.org/airnow/2019/20190528/HourlyAQObs_2019052815.dat)
EARLIEST_DATA_FILE_DATETIME = datetime.datetime(2019, 5, 28, 15, 0, 0)

SECONDS_TO_PAUSE_BETWEEN_DOWNLOADS = 0.5

NUM_TRAILING_DAYS = 30

MIRROR_TIME_PERIOD_SECS = 60 * 30

STAT_HOSTNAME = 'hal21'
STAT_SHORTNAME = 'airnow-hourly-aqi-download'

In [ ]:
Stat.set_service('Airnow Hourly AQI - Download')

In [ ]:
# make sure data directories exist and are readable to all
os.makedirs(AirnowCommon.HOURLY_AQI_DAT_DIRECTORY, exist_ok=True)
os.makedirs(AirnowCommon.HOURLY_AQI_JSON_DIRECTORY, exist_ok=True)
os.chmod(AirnowCommon.HOURLY_AQI_DIRECTORY, stat.S_IRUSR | stat.S_IWUSR | stat.S_IXUSR | stat.S_IRGRP | stat.S_IXGRP | stat.S_IROTH | stat.S_IXOTH)
os.chmod(AirnowCommon.HOURLY_AQI_DAT_DIRECTORY, stat.S_IRUSR | stat.S_IWUSR | stat.S_IXUSR | stat.S_IRGRP | stat.S_IXGRP | stat.S_IROTH | stat.S_IXOTH)
os.chmod(AirnowCommon.HOURLY_AQI_JSON_DIRECTORY, stat.S_IRUSR | stat.S_IWUSR | stat.S_IXUSR | stat.S_IRGRP | stat.S_IXGRP | stat.S_IROTH | stat.S_IXOTH)

In [ ]:
def mirror_timestamp(timestamp):
    filename = timestamp.strftime('%Y%m%d%H.dat')
    src = AirnowCommon.directory_from_date(timestamp) + '/' + timestamp.strftime('HourlyAQObs_%Y%m%d%H.dat')
    dest = AirnowCommon.HOURLY_AQI_DAT_DIRECTORY + '/' + timestamp.strftime('%Y%m%d%H.dat')
    print(src)
    print(dest)
    (is_new, message, status_code) = AirnowCommon.mirror_airnow_file(src, dest)

    if is_new:
        Stat.info(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    else:
        if status_code == 304:
            print(message) # simply do a print here to reduce noise sent to stat.createlab.org
            # Stat.info(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        elif status_code < 400:
            Stat.info(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        else:
            Stat.warning(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)


#mirror_timestamp(EARLIEST_DATA_FILE_DATETIME)

In [ ]:
def compute_first_date_to_check():
    files = glob.glob(AirnowCommon.HOURLY_AQI_DAT_DIRECTORY + '/[0-9]*.dat')
    if len(files) == 0:
        return EARLIEST_DATA_FILE_DATETIME
    last_file = sorted(files)[-1]
    last_date = datetime.datetime.strptime(last_file, AirnowCommon.HOURLY_AQI_DAT_DIRECTORY + "/%Y%m%d%H.dat")
    Stat.debug('Most recently mirrored data file is %s (%s)' % (last_file, last_date), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    first_date_to_check = last_date - datetime.timedelta(days=NUM_TRAILING_DAYS)
    Stat.debug('Checking for updates starting with date %s' % (first_date_to_check), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    sys.stdout.flush()
    return first_date_to_check


#compute_first_date_to_check()

In [ ]:
def mirror():
    starting_timestamp = datetime.datetime.now().timestamp()
    start = compute_first_date_to_check()
    now = datetime.datetime.utcnow()
    timestamps_to_mirror = list(rrule.rrule(rrule.HOURLY, dtstart=start, until=now))

    Stat.info('Mirroring %d data files, starting with %s... (up-to-date files will not be logged here)' % (len(timestamps_to_mirror), start), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    for timestamp in timestamps_to_mirror:
        mirror_timestamp(timestamp)
        time.sleep(SECONDS_TO_PAUSE_BETWEEN_DOWNLOADS)

    elapsed_seconds = datetime.datetime.now().timestamp() - starting_timestamp
    Stat.up('Done mirroring %d data files (elapsed time: %d seconds)' % (len(timestamps_to_mirror), elapsed_seconds), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME, valid_for_secs=MIRROR_TIME_PERIOD_SECS*1.5)

def mirror_forever():
    while True:
        mirror()
        sleep_until_next_period(MIRROR_TIME_PERIOD_SECS, 10*60)  # start at 10 minutes after the hour

mirror_forever()